In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Read both dataset and pre-process together
data_train = pd.read_csv("E:/Trinity/Machine Learning/Kaggle/tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv")
data_test = pd.read_csv("E:/Trinity/Machine Learning/Kaggle/tcdml1920-income-ind/tcd ml 2019-20 income prediction test (without labels).csv")

In [3]:
data = pd.concat([data_train, data_test], sort=False)

In [4]:
data.head()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
0,1,1997.0,0,41.0,Belarus,1239930,steel workers,Bachelor,0,Blond,193,61031.94416
1,2,1996.0,other,41.0,Singapore,1603504,safe event coordinator,Master,0,Black,186,91001.32764
2,3,2018.0,other,28.0,Norway,1298017,receivables/payables analyst,PhD,1,Brown,170,157982.17670
3,4,2006.0,other,33.0,Cuba,751903,fleet assistant,No,1,Black,171,45993.75793
4,5,2010.0,female,46.0,United Arab Emirates,95389,lead trainer,0,0,Blond,188,38022.16217


In [5]:
len(data)

185223

In [6]:
data = data.drop("Instance", axis=1)     # Drop column that has no relevance

In [7]:
data.isnull().sum()                      # Check dataframe for NULL values

Year of Record         736
Gender               12294
Age                    773
Country                  0
Size of City             0
Profession             517
University Degree    12227
Wears Glasses            0
Hair Color           12158
Body Height [cm]         0
Income in EUR        73230
dtype: int64

In [8]:
# Rename all multi-word column names to single words for easier access

data = data.rename(index=str, columns={"Body Height [cm]" : "Height"})
data = data.rename(index=str, columns={"Year of Record" : "YearOfRecord"})
data = data.rename(index=str, columns={"Size of City" : "SizeOfCity"})
data = data.rename(index=str, columns={"University Degree": "UniversityDegree"})
data = data.rename(index=str, columns={"Wears Glasses" : "WearsGlasses"})
data = data.rename(index=str, columns={"Hair Color" : "HairColor"})
data = data.rename(index=str, columns={"Income in EUR" : "Income"})

In [9]:
data.YearOfRecord.unique()

array([1997., 1996., 2018., 2006., 2010., 1991., 1982., 2008., 2015.,
       2019., 1981., 1989., 2003., 1987., 1995., 1992., 1980., 1983.,
       2011., 2014., 1985., 2005., 1998., 2001., 2017., 2007., 1988.,
       1994., 2012., 2009., 2013., 2004., 1999., 2002., 1993., 2000.,
       1990., 2016.,   nan, 1986., 1984.])

In [10]:
data.Gender.unique()

array(['0', 'other', 'female', 'male', nan, 'unknown'], dtype=object)

In [11]:
# Data imputation

data['Gender'] = data['Gender'].replace('0', "other")                   
data['Gender'] = data['Gender'].replace('unknown', pd.np.nan) 

In [12]:
data.Age.unique()

array([ 41.,  28.,  33.,  46.,  60.,  71.,  36.,  43.,  16.,  51.,  15.,
        22.,  32.,  40.,  31.,  27.,  25.,  35.,  63.,  38.,  34.,  20.,
        23.,  14.,  37.,  62.,  52.,  47.,  24.,  26.,  50.,  49.,  18.,
        19.,  39.,  66.,  21.,  44.,  29.,  64.,  65.,  53.,  78.,  42.,
        58.,  17.,  30.,  79.,  48.,  56.,  83.,  54.,  74.,  87.,  90.,
        67.,  45.,  77.,  73.,  55.,  92.,  80.,  72.,  57.,  70.,  59.,
        61.,  82.,  69.,  98.,  nan,  94.,  68.,  81.,  97.,  76.,  75.,
        88.,  91.,  96.,  84.,  89., 107.,  85.,  86., 100., 110.,  95.,
        99.,  93., 104., 102., 101., 105., 111., 108., 103., 109., 106.,
       115., 120., 126., 113.])

In [13]:
data.UniversityDegree.unique()

array(['Bachelor', 'Master', 'PhD', 'No', '0', nan], dtype=object)

In [14]:
# Label Encoding for University Degree giving different weights to different degrees
data['UniversityDegree'] = data['UniversityDegree'].replace('PhD', 4) 
data['UniversityDegree'] = data['UniversityDegree'].replace('Master', 3) 
data['UniversityDegree'] = data['UniversityDegree'].replace('Bachelor', 2) 
data['UniversityDegree'] = data['UniversityDegree'].replace('No', 0) 
data['UniversityDegree'] = data['UniversityDegree'].replace(pd.np.nan, 0) 

In [15]:
data.WearsGlasses.unique()

array([0, 1], dtype=int64)

In [16]:
data.HairColor.unique()

array(['Blond', 'Black', 'Brown', nan, 'Red', 'Unknown', '0'],
      dtype=object)

In [17]:
data['HairColor'] = data['HairColor'].replace('0', pd.np.nan) 
data['HairColor'] = data['HairColor'].replace('Unknown', pd.np.nan)

In [18]:
# Function to implement Target Encoding
def calc_smooth_mean(df, by, on, m):
    mean = df[on].mean()                                 # Compute the global mean
    agg = df.groupby(by)[on].agg(['count', 'mean'])      # Compute number of values & mean of each group
    counts = agg['count']
    means = agg['mean']

    smooth = (counts * means + m * mean) / (counts + m)  # Compute the "smoothed" means
    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

In [19]:
# Target Encoding on Country
data['Country'] = calc_smooth_mean(data, 'Country', 'Income', 2)

In [20]:
# Target Encoding on Profession (More weight given for column with null values)
data['Profession'] = calc_smooth_mean(data, 'Profession', 'Income', 50)

In [21]:
data.head()

,YearOfRecord,Gender,Age,Country,SizeOfCity,Profession,UniversityDegree,WearsGlasses,HairColor,Height,Income
0,1997.0,other,41.0,33188.806425,1239930,134565.216092,2,0,Blond,193,61031.94416
1,1996.0,other,41.0,56341.514457,1603504,129098.312715,3,0,Black,186,91001.32764
2,2018.0,other,28.0,87145.867608,1298017,115618.688793,4,1,Brown,170,157982.17670
3,2006.0,other,33.0,100619.026294,751903,83784.547676,0,1,Black,171,45993.75793
4,2010.0,female,46.0,42837.800904,95389,93117.076103,0,0,Blond,188,38022.16217


In [22]:
data.isnull().sum()

YearOfRecord          736
Gender              23489
Age                   773
Country                10
SizeOfCity              0
Profession            538
UniversityDegree        0
WearsGlasses            0
HairColor           13276
Height                  0
Income              73230
dtype: int64

In [23]:
data.head()

,YearOfRecord,Gender,Age,Country,SizeOfCity,Profession,UniversityDegree,WearsGlasses,HairColor,Height,Income
0,1997.0,other,41.0,33188.806425,1239930,134565.216092,2,0,Blond,193,61031.94416
1,1996.0,other,41.0,56341.514457,1603504,129098.312715,3,0,Black,186,91001.32764
2,2018.0,other,28.0,87145.867608,1298017,115618.688793,4,1,Brown,170,157982.17670
3,2006.0,other,33.0,100619.026294,751903,83784.547676,0,1,Black,171,45993.75793
4,2010.0,female,46.0,42837.800904,95389,93117.076103,0,0,Blond,188,38022.16217


In [24]:
'''
data.drop("YearOfRecord", axis=1)
data.drop("Country", axis=1)
data.drop("WearsGlasses", axis=1)
'''



'\ndata.drop("YearOfRecord", axis=1)\ndata.drop("Country", axis=1)\ndata.drop("WearsGlasses", axis=1)\n'

In [25]:
# One Hot Encoding 
data1 = pd.get_dummies(data, columns=["Gender"], drop_first = True)
#data1 = pd.get_dummies(data1, columns=["Country"], drop_first = True)
#data1 = pd.get_dummies(data1, columns=["Profession"], drop_first = True)
#data1 = pd.get_dummies(data1, columns=["UniversityDegree"], drop_first = True)
data1 = pd.get_dummies(data1, columns=["HairColor"], drop_first = True)

In [26]:
pd.set_option('display.max_columns', 100)
data1

,YearOfRecord,Age,Country,SizeOfCity,Profession,UniversityDegree,WearsGlasses,Height,Income,Gender_male,Gender_other,HairColor_Blond,HairColor_Brown,HairColor_Red
0,1997.0,41.0,33188.806425,1239930,134565.216092,2,0,193,61031.94416,0,1,1,0,0
1,1996.0,41.0,56341.514457,1603504,129098.312715,3,0,186,91001.32764,0,1,0,0,0
2,2018.0,28.0,87145.867608,1298017,115618.688793,4,1,170,157982.17670,0,1,0,1,0
3,2006.0,33.0,100619.026294,751903,83784.547676,0,1,171,45993.75793,0,1,0,0,0
4,2010.0,46.0,42837.800904,95389,93117.076103,0,0,188,38022.16217,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73225,2012.0,37.0,87145.867608,62975,96669.318540,2,0,203,NaN,1,0,0,1,0
73226,2016.0,42.0,72369.724328,46819,110315.103844,0,0,181,NaN,0,1,0,0,0
73227,2019.0,50.0,46320.066299,525072,90328.171180,3,1,162,NaN,0,1,0,1,0
73228,1980.0,54.0,252230.618903,915892,85715.746612,3,1,162,NaN,0,0,0,1,0


In [27]:
data1.isnull().sum()

YearOfRecord          736
Age                   773
Country                10
SizeOfCity              0
Profession            538
UniversityDegree        0
WearsGlasses            0
Height                  0
Income              73230
Gender_male             0
Gender_other            0
HairColor_Blond         0
HairColor_Brown         0
HairColor_Red           0
dtype: int64

In [28]:
data1.tail()

,YearOfRecord,Age,Country,SizeOfCity,Profession,UniversityDegree,WearsGlasses,Height,Income,Gender_male,Gender_other,HairColor_Blond,HairColor_Brown,HairColor_Red
73225,2012.0,37.0,87145.867608,62975,96669.318540,2,0,203,NaN,1,0,0,1,0
73226,2016.0,42.0,72369.724328,46819,110315.103844,0,0,181,NaN,0,1,0,0,0
73227,2019.0,50.0,46320.066299,525072,90328.171180,3,1,162,NaN,0,1,0,1,0
73228,1980.0,54.0,252230.618903,915892,85715.746612,3,1,162,NaN,0,0,0,1,0
73229,2017.0,41.0,135680.229754,62607,105817.832455,3,0,197,NaN,0,1,0,1,0


In [29]:
X_train = data1[0:len(data_train)]          # Dataframe again split into training and test dataset

In [30]:
len(X_train)

111993

In [31]:
# Fill null values 
X_train["YearOfRecord"].fillna((X_train["YearOfRecord"].mean()), inplace=True )
X_train["Age"].fillna((X_train["Age"].mean()), inplace=True )
X_train["Profession"].fillna((X_train["Profession"].mean()), inplace=True )

C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [32]:
X_train.isnull().sum()

YearOfRecord        0
Age                 0
Country             0
SizeOfCity          0
Profession          0
UniversityDegree    0
WearsGlasses        0
Height              0
Income              0
Gender_male         0
Gender_other        0
HairColor_Blond     0
HairColor_Brown     0
HairColor_Red       0
dtype: int64

In [33]:
Y_train = X_train[["Income"]]                # Split table into predictors and response
X_train = X_train.drop("Income", axis=1)

In [34]:
# Split Training Data into training and holout data 

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
X_training, X_holdOut, Y_training, Y_holdOut = train_test_split(X_train, Y_train, train_size=0.9, random_state=100)

In [35]:
# Apply RandomForestRegressor on training data

from sklearn.ensemble import RandomForestRegressor
LR = RandomForestRegressor(n_estimators=1000, random_state=100)
#LR = RandomForestRegressor(max_depth=4, max_features='log2', min_samples_leaf=0.1, n_estimators=400, random_state=100)

In [36]:
model = LR.fit(X_training, Y_training)        # Fit the RFR model

C:\Users\Lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [38]:
# Predict the holdout data
from sklearn.metrics import mean_squared_error, r2_score
ypred = model.predict(X_holdOut)

In [39]:
# Calculate the different metrics
import math

mse = mean_squared_error(Y_holdOut, ypred)
rmse = math.sqrt(mse)
rmse

60526.990096858084

In [40]:
####################################### Prediction Time####################################

X_test = data1[len(data_train):]
X_test = X_test.drop("Income", axis=1)

In [ ]:
X_test.isnull().sum()

In [ ]:
# Fill all null values with mean
X_test["YearOfRecord"].fillna((X_test["YearOfRecord"].mean()), inplace=True )
X_test["Age"].fillna((X_test["Age"].mean()), inplace=True )
X_test["Profession"].fillna((X_test["Profession"].mean()), inplace=True )
X_test["Country"].fillna((X_test["Country"].mean()), inplace=True )

In [ ]:
X_test.isnull().sum()

In [ ]:
Y_pred = model.predict(X_test)      # Predict response for out of sample data

In [ ]:
# Write predicted data to CSV
Y_pred = pd.DataFrame(Y_pred)
Y_pred.to_csv("E:/Trinity/Machine Learning/Kaggle/tcdml1920-income-ind/submission.csv", sep=',', index=False, header=True)